In [ ]:
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/vsc.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/trust_ncg.py
!wget https://raw.githubusercontent.com/profteachkids/CHE2064/master/che_tools.py
!pip install DotMap

In [1]:
from dotmap import DotMap
import jax.numpy as jnp
import jax
import vsc
import trust_ncg as tncg
import che_tools as che

In [2]:
p = che.Props(['Ethanol', 'Isopropanol', 'Water'])

In [3]:
def model(c):
    c=DotMap(c)
    x=che.qtox(c.Vq)
    y=che.qtox(c.Lq)
    z=che.qtox(c.Fq)
    V = x * c.Vtot
    L = y * c.Ltot
    F = z * c.Ftot
    mass_balance = (F - V - L)
    mass_balance = (mass_balance/jnp.sum(F))**2

    FH = p.Hmix( (1.-c.q)*F, c.q*F, c.FT)
    VH = p.Hmix(V, 0, c.flashT)
    LH = p.Hmix(0, L, c.flashT)
    energy_balance = (FH - VH - LH)
    energy_balance = (energy_balance/jnp.sum(FH))**2

    fugacity = (x*p.Pvap(c.flashT) - y*c.flashP)
    fugacity = (fugacity/jnp.sum(y*c.flashP))**2

    return jnp.sum(mass_balance + energy_balance + fugacity)

In [4]:
s=DotMap()
s.Ftot=10
s.Fq = che.xtoq([1/3, 1/3, 1/3])
s.FT = 350
s.flashP= 101325
s.q = 0.5 #fraction liquid

v=DotMap()
v.Vq = che.xtoq([0.4, 0.35, 0.25])
v.Lq = che.xtoq([0.25, 0.35, 0.4])
v.flashT = 300
v.Vtot = s.Ftot/2
v.Ltot = s.Ftot/2

[Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>, Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>, Traced<ShapedArray(float64[], weak_type=True):JaxprTrace(level=-1/1)>]


/Users/carlosco/opt/anaconda3/envs/pytorch/lib/python3.8/site-packages/jax/lib/xla_bridge.py:130: UserWarning: No GPU/TPU found, falling back to CPU.
  warnings.warn('No GPU/TPU found, falling back to CPU.')


In [5]:
c=vsc.VSC(v,s)
modelf = jax.jit(c.transform(model))
x_min, f = tncg.minimize(modelf,c.x, verbosity=1)

0:4, f: 0.3182956617493208
1:3, f: 0.2951542974058736
2:5, f: 0.2895570764862791
3:4, f: 0.2805715753193567
4:5, f: 0.25219124942674875
5:4, f: 0.1934232645111691
6:2, f: 0.1934232645111691
7:2, f: 0.1934232645111691
8:4, f: 0.17067381117819186
9:4, f: 0.15238815883140183
10:4, f: 0.13187458640068903
11:5, f: 0.08792602805737397
12:6, f: 0.012659903933506063
13:6, f: 0.0005386452972950885
14:6, f: 3.525986138217572e-06
15:6, f: 4.6408293175236376e-10
Final results:
f: 8.83358843701275e-18
x: [ 4.01011698e-01  3.36410535e-01  5.28612421e+00 -4.14090787e-01
 -3.21380334e-01  4.71387582e+00  3.59966484e+02]


In [6]:
v_sol = c.xtoc(x_min)
df=vsc.todf(v_sol)

